# Select grids with VLLM

## Goal

Can I use VLLM to select the correct grid answer?

## Configuration

In [ ]:
class cfg:
    model_path: str = "/home/gbarbadillo/data/Qwen2-0.5B-arc"
    max_model_len: int = 8192 #61000 for phi-3
    solutions_filepath: str = '/mnt/hdd0/MEGA/AI/22_Kaggle/arc24/scripts/submission_x512.json'
    dataset_filepath: str = '/mnt/hdd0/Kaggle/arc24/data/arc-agi_evaluation_challenges.json'

## Imports

In [ ]:
import sys
import os
import json
import time
import textwrap
from vllm import LLM, SamplingParams
from transformers import AutoTokenizer
import matplotlib.pyplot as plt
import matplotlib as mpl

sys.path.append(os.path.realpath('../scripts/'))

from evaluation import (
    load_arc_data_with_solutions,
    evaluate
)
from inference import (
    clear_vllm_gpu_memory,
    SimplePromptCreator,
    GridCodeBlockEncoder,
    MinimalGridEncoder
)
from arc24.prompting import (
    pretty_print_prompt,
    system_prompt,
    prompt_template,
    answer_template,
    remove_assistant_ending
)

plt.plot()
plt.close('all')
plt.rcParams["figure.figsize"] = (25, 4)
mpl.rcParams['lines.linewidth'] = 3
mpl.rcParams['font.size'] = 16

## Load model

In [ ]:
llm = LLM(model=cfg.model_path,
            trust_remote_code=True,
            dtype='half',
            tensor_parallel_size=2, # to use 2 gpus
            max_model_len=cfg.max_model_len,
            #kv_cache_dtype='fp8_e5m2', I have disabled kv cache quantization because it is hurtful
            enforce_eager=True, # without this 13.9GB of memory is used on each GPU, with this is 13.3GB,
            disable_log_stats=True,
            )
tokenizer = AutoTokenizer.from_pretrained(cfg.model_path)

## Prepare prompts

In [ ]:
with open(cfg.solutions_filepath, 'r') as f:
    solutions = json.load(f)
ground_truth = load_arc_data_with_solutions(cfg.dataset_filepath)

In [ ]:
grid_encoder = GridCodeBlockEncoder(MinimalGridEncoder())

In [ ]:
def create_prompt(task, test_idx, grid):
    train_samples = [{key: grid_encoder.to_text(grid) for key, grid in sample.items()} for sample in task['train']]
    user_message = prompt_template.render(train_samples=train_samples,
                                            test_input=grid_encoder.to_text(task['test'][test_idx]['input']))
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message},
                {"role": "assistant", "content": answer_template.render(test_output=grid_encoder.to_text(grid))}]
    # TODO: add start of assistant reply
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    prompt = remove_assistant_ending(prompt, cfg.model_path)
    return prompt

In [ ]:
task_id = list(solutions.keys())[0]
test_idx = 0
prompt = create_prompt(ground_truth[task_id], test_idx=test_idx, grid=ground_truth[task_id]['test'][test_idx]['output'])
print(len(tokenizer.tokenize(prompt)))
pretty_print_prompt(prompt, default_color='white')

## Compute likelihood of each prompt

- [Github issue: Add efficient interface for evaluating probabilities of fixed prompt-completion pairs](https://github.com/vllm-project/vllm/issues/5234)
- [prompt_logprobs](https://docs.vllm.ai/en/latest/dev/sampling_params.html) – Number of log probabilities to return per prompt token.

In [ ]:
def print_attributes(obj, indent=0):
    """
    Recursively prints the attributes of an object and their values.
    
    :param obj: The object to inspect.
    :param indent: The current level of indentation (used for recursive calls).
    """
    spacing = ' ' * indent
    print(f"{spacing}{obj.__class__.__name__}:")

    # Get the attributes of the object
    for attr in dir(obj):
        # Ignore special attributes/methods
        if not attr.startswith('__') and not callable(getattr(obj, attr)):
            value = getattr(obj, attr)
            if hasattr(value, '__dict__'):
                print(f"{spacing}  {attr}:")
                print_attributes(value, indent + 4)
            # elif isinstance(value, list):
            #     print(f"{spacing}  {attr}:")
            #     for item in value:
            #         print(f"{spacing}    -")
            #         print_attributes(item, indent + 4)
            else:
                value = str(value)[:100]
                print(f"{spacing}  {attr}: {value}")

In [ ]:
raise

In [ ]:
sampling_params = SamplingParams(n=1, temperature=0.0, max_tokens=1)
ret = llm.generate(prompt, sampling_params, use_tqdm=False)
print_attributes(ret[0])

In [ ]:
print_attributes(ret[0].outputs[0])

I can generate a single extra token very fast, but if I require the `prompt_logprobs` I get an OOM error.

In [ ]:
sampling_params = SamplingParams(n=1, temperature=0.0, max_tokens=1, prompt_logprobs=1)
ret = llm.generate(prompt[:100], sampling_params, use_tqdm=False)
print_attributes(ret[0])

In [ ]:
ret[0].prompt_logprobs

It is able to work with a small prompt though, and it outputs what we need. However for a prompt length between 2000 and 3000 characters the OOM happens.

Thus I can not use this feature with VLLM, I would have to create my own implementation if I really want to do it.

In [ ]:
sampling_params = SamplingParams(n=1, temperature=0.0, max_tokens=1, prompt_logprobs=1)
ret = llm.generate(prompt[:3000], sampling_params, use_tqdm=False)
print_attributes(ret[0])

## Get likelihood of the generated output

I can try to use the likelihood of the already generated outputs. It is not the same because the inputs for each generation will be different. But maybe is enough...

In [ ]:
sampling_params = SamplingParams(n=1, temperature=0.0, max_tokens=10, logprobs=1)
ret = llm.generate(prompt[:-9], sampling_params, use_tqdm=False)
print_attributes(ret[0])

In [ ]:
print_attributes(ret[0].outputs[0])

I believe that I can simply save the `cumulative_logprob` and the number of generated tokens.

In [ ]:
sampling_params = SamplingParams(n=1, temperature=0.0, max_tokens=10, logprobs=0)
ret = llm.generate(prompt[:-9], sampling_params, use_tqdm=False)
print_attributes(ret[0].outputs[0])

In [ ]:
dict(cumulative_logprob=ret[0].outputs[0].cumulative_logprob, n_tokens=len(ret[0].outputs[0].token_ids))

I could compute a mean cumulative_logprob to compare the different answers.

## Clean

In [ ]:
del llm.llm_engine.model_executor
del llm
clear_vllm_gpu_memory()

## TODO

- [ ] Print all the elements in the response.